### Imports

In [1]:
import psycopg2
import pandas as pd

In [2]:
def create_database():
    #connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=**** password=****
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    #create sparkify database with UTF8 encoding
    cur.execute("DROP DATABASE IF EXISTS accounts")
    cur.execute("CREATE DATABASE accounts")
    
    #close connection to default database
    conn.close()
    
    #connect to sparkify database
    conn = psycopg2.connect("host=127.0.0.1 dbname=accounts user=**** password=****")
    cur = conn.cursor()
    
    return cur, conn

In [3]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

### Accounts Country

In [5]:
AccountsCountry = pd.read_csv("Wealth Data/Wealth-AccountsCountry.csv", nrows=147)

In [6]:
AccountsCountry.columns

Index(['Code', 'Long Name', 'Income Group', 'Region', 'Lending category',
       'Other groups', 'Currency Unit', 'Latest population census',
       'Latest household survey', 'Special Notes',
       'National accounts base year', 'National accounts reference year',
       'System of National Accounts', 'SNA price valuation',
       'Alternative conversion factor', 'PPP survey years',
       'Balance of Payments Manual in use', 'External debt Reporting status',
       'System of trade', 'Government Accounting concept',
       'IMF data dissemination standard',
       'Source of most recent Income and expenditure data',
       'Vital registration complete', 'Latest agricultural census',
       'Latest industrial data', 'Latest trade data',
       'Latest water withdrawal data', '2-alpha code', 'WB-2 code',
       'Table Name', 'Short Name'],
      dtype='object')

In [7]:
selected_columns = ["Code", "Short Name", "Table Name", "Long Name", "Currency Unit"]
AccountsCountry_clean = AccountsCountry[selected_columns]

In [8]:
AccountsCountry_clean = AccountsCountry_clean.drop_duplicates(subset=["Code"])

In [9]:
AccountsCountry_clean.head()

,Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram
3,AUS,Australia,Australia,Commonwealth of Australia,Australian dollar
4,AUT,Austria,Austria,Republic of Austria,Euro


In [10]:
print(AccountsCountry.shape)
print(AccountsCountry_clean.shape)

(147, 31)
(146, 5)


In [11]:
AccountsCountry_clean.head()

,Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram
3,AUS,Australia,Australia,Commonwealth of Australia,Australian dollar
4,AUT,Austria,Austria,Republic of Austria,Euro


### Acounts Data

In [12]:
AccountsData = pd.read_csv("Wealth Data/Wealth-AccountData.csv")

In [13]:
AccountsData.shape

(7592, 28)

In [14]:
AccountsData.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]'],
      dtype='object')

In [15]:
AccountsData = AccountsData[['Country Name', 'Country Code', 'Series Name', 'Series Code',
                           '1995 [YR1995]', '2000 [YR2000]', '2005 [YR2005]', '2010 [YR2010]',
                           '2014 [YR2014]']]

In [16]:
AccountsData.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '2000 [YR2000]', '2005 [YR2005]', '2010 [YR2010]',
       '2014 [YR2014]'],
      dtype='object')

In [31]:
AccountsData_clean = AccountsData.replace('..', 0)

### Accounts Series

In [17]:
AccountsSeries = pd.read_csv("Wealth Data/Wealth-AccountSeries.csv")

In [18]:
AccountsSeries.columns

Index(['Code', 'Indicator Name', 'Long definition', 'Source', 'Topic',
       'Unit of measure', 'Periodicity', 'Reference period',
       'Statistical concept and methodology', 'Previous Indicator Code',
       'Previous Indicator Name'],
      dtype='object')

In [19]:
AccountsSeries = AccountsSeries[['Code',"Topic","Indicator Name","Long definition"]]

In [20]:
AccountsSeries.head()

,Code,Topic,Indicator Name,Long definition
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$),Human capital is computed as the present value...
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...
2,NW.HCA.FEMP.PC,Human capital,"Human capital per capita, employed female (con...",Human capital is computed as the present value...
3,NW.HCA.MEMP.PC,Human capital,"Human capital per capita, employed male (const...",Human capital is computed as the present value...
4,NW.HCA.FEMA.PC,Human capital,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...


In [21]:
cur, conn = create_database()

In [22]:
songplay_table_create = ("""CREATE TABLE IF NOT EXISTS accountscountry(
code VARCHAR PRIMARY KEY,
short_name VARCHAR,
table_name VARCHAR,
long_name VARCHAR,
currency_unit VARCHAR)
""")

In [23]:
cur.execute(songplay_table_create)
conn.commit()


In [24]:
accounts_data_table_create = ("""CREATE TABLE IF NOT EXISTS accountsdata(
country_name VARCHAR,
country_code VARCHAR,
series_name VARCHAR,
series_code VARCHAR,
year_1995 numeric,
year_2000 numeric,
year_2005 numeric,
year_2010 numeric,
year_2014 numeric
)""")
cur.execute(accounts_data_table_create)
conn.commit()

In [25]:
accountseries_data_table_create = (""" CREATE TABLE IF NOT EXISTS accountseries(
code VARCHAR,
topic VARCHAR,
indicator_name VARCHAR,
long_definition VARCHAR
)""")
cur.execute(accountseries_data_table_create)
conn.commit()

In [26]:
accounts_country_table_insert = ("""INSERT INTO accountscountry (
code,
short_name,
table_name,
long_name,
currency_unit
) VALUES (%s, %s, %s, %s, %s)

""")

In [27]:
for i, row in AccountsCountry_clean.iterrows():
    cur.execute(accounts_country_table_insert, list(row))

In [28]:
conn.commit()

In [29]:
accounts_data_table_insert = ("""INSERT INTO accountsdata(
country_name,
country_code,
series_name,
series_code,
year_1995,
year_2000,
year_2005,
year_2010,
year_2014
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

In [36]:
for i, row in AccountsData_clean.iterrows():
    cur.execute(accounts_data_table_insert, list(row))

In [37]:
conn.commit()

In [38]:
accountseries_data_table_insert = ("""INSERT INTO accountseries(
code,
topic,
indicator_name,
long_definition
) VALUES (%s, %s, %s, %s)
""")

In [39]:
for i, row in AccountsSeries.iterrows():
    cur.execute(accountseries_data_table_insert, list(row))

In [40]:
conn.commit()